In [101]:
def split_A_by_annotator(in_, out_):
    annotator_id = '0'
    
    with open(in_) as input_file, open(out_, 'w') as output_file:
        for line in input_file:
            line = line.strip()
            if line.startswith('S'):
                S = line
                output_file.write(f"{S}\n")
            elif line.startswith('A'):
                A = line
                if line.split('|||')[-1] == annotator_id:
                    output_file.write(f"{A}\n")
                else:
                    annotator_id = line.split('|||')[-1]
                    output_file.write('\n')
                    output_file.write(f"{S}\n")
                    output_file.write(f"{A}\n")
            else:
                output_file.write('\n')

In [102]:
split_A_by_annotator("fce.train.gold.bea19.m2", "fce.train.gold.bea19_.m2")

In [103]:
split_A_by_annotator("test.m2", "test_.m2")

In [104]:
import sys
import re

def m2_to_csv(in_, out_):
    words = []
    corrected = []
    sid = eid = 0
    prev_sid = prev_eid = -1
    pos = 0
    
    with open(in_) as input_file, open(out_, 'w') as output_file:
        for line in input_file:
            line = line.strip()
            if line.startswith('S'):
                line = line[2:]
                words = line.split()
                corrected = ['<S>'] + words[:]
                output_file.write(line + '\n')
            elif line.startswith('A'):
                line = line[2:]
                info = line.split("|||")
                sid, eid = info[0].split()
                sid = int(sid) + 1; eid = int(eid) + 1;
                error_type = info[1]
                if error_type == "Um":
                    continue
                for idx in range(sid, eid):
                    corrected[idx] = ""
                if sid == eid:
                    if sid == 0: continue	# Originally index was -1, indicating no op
                    if sid != prev_sid or eid != prev_eid:
                        pos = len(corrected[sid-1].split())
                    cur_words = corrected[sid-1].split()
                    cur_words.insert(pos, info[2])
                    pos += len(info[2].split())
                    corrected[sid-1] = " ".join(cur_words)
                else:
                    corrected[sid] = info[2]
                    pos = 0
                prev_sid = sid
                prev_eid = eid
            else:
                target_sentence = ' '.join([word for word in corrected if word != ""])
                assert target_sentence.startswith('<S>'), '(' + target_sentence + ')'
                target_sentence = target_sentence[4:]
                output_file.write(target_sentence + '\n\n')
                prev_sid = -1
                prev_eid = -1
                pos = 0


In [105]:
m2_to_csv("/home/neko/Downloads/fce.train.gold.bea19_.m2", "/home/neko/Downloads/out.txt")

In [106]:
m2_to_csv("/home/neko/Downloads/test_.m2", "/home/neko/Downloads/out1.txt")